Notebook adapted from https://www.kaggle.com/bguberfain/naive-keras

Setup the kernel

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import downscale_local_mean
from os.path import join
from tqdm.notebook import tqdm
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_addons as tfa
import os
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator

In [7]:
data_dir = 'G:/Github/Caravan_challenge/data/train/'
mask_dir = 'G:/Github/Caravan_challenge/data/train_masks/'
test_dir = 'G:/Github/Caravan_challenge/data/test/'
all_images = os.listdir(data_dir)
image_size = 224

Setup data input

In [8]:
train_images, validation_images = train_test_split(all_images, train_size=0.8, test_size=0.2)

In [21]:
# generator that we will use to read the data from the directory
def data_gen_small(data_dir, mask_dir, images, batch_size, dims):
        """
        data_dir: where the actual images are kept
        mask_dir: where the actual masks are kept
        images: the filenames of the images we want to generate batches from
        batch_size: self explanatory
        dims: the dimensions in which we want to rescale our images, tuple
        """
        while True:
            ix = np.random.choice(np.arange(len(images)), batch_size)
            imgs = []
            labels = []
            for i in ix:
                # images
                original_img = load_img(data_dir + images[i])
                resized_img = original_img.resize(dims)
                array_img = img_to_array(resized_img)/255
                imgs.append(array_img)
                
                # masks
                original_mask = load_img(mask_dir + images[i].split(".")[0] + '_mask.gif')
                resized_mask = original_mask.resize(dims)
                array_mask = img_to_array(resized_mask)/255
                labels.append(array_mask[:, :, 0])
            imgs = np.array(imgs)
            labels = np.array(labels)
            yield imgs, labels.reshape(-1, dims[0], dims[1], 1)

In [22]:
train_gen = data_gen_small(data_dir, mask_dir, train_images, 5, (image_size, image_size))
val_gen = data_gen_small(data_dir, mask_dir, validation_images, 5, (image_size, image_size))

Define our own dice_coef (F1) for the images

In [23]:
# Now let's use Tensorflow to write our own dice_coeficcient metric
def dice_coef(y_true, y_pred):
    smooth = 1e-5
    
    y_true = tf.round(tf.reshape(y_true, [-1]))
    y_pred = tf.round(tf.reshape(y_pred, [-1]))
    
    isct = tf.reduce_sum(y_true * y_pred)
    
    return 2 * isct / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred))

Set up a simple CNN model

In [24]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras import Sequential
from tensorflow.keras.losses import BinaryCrossentropy

model = Sequential()
model.add( Conv2D(16, 3, activation='relu', padding='same', input_shape=(image_size, image_size, 3)) )
model.add( Conv2D(32, 3, activation='relu', padding='same') )
model.add( Conv2D(1, 5, activation='sigmoid', padding='same') )

Complile model

In [25]:
model.compile(optimizer = 'Adam', loss = BinaryCrossentropy(), metrics=['accuracy',dice_coef])

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 224, 224, 16)      448       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 224, 224, 32)      4640      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 224, 224, 1)       801       
Total params: 5,889
Trainable params: 5,889
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_gen, epochs=24, steps_per_epoch=212, validation_data = val_gen,  verbose=2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 212 steps
Epoch 1/24


In [ ]:
training_loss = model.history.history['loss']
test_loss = model.history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
training_loss = model.history.history['accuracy']
test_loss = model.history.history['val_accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')